In [ ]:
# import packages

In [ ]:
from pandas.io.json import json_normalize
import json
from pandasql import sqldf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import io
import requests
import json
import base64
import datetime
from urllib.parse import urlencode

In [ ]:
# client id

In [ ]:
client_id = '6e24789ec7874259a220f747fcd7e93e'
client_secret = '9bd8ff1ad2f34ef6a87f00124be070b2'

In [ ]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"



    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def getClientCreds(self):
        '''Returns b64 encoded string'''
        client_id = self.client_id
        client_secret = self.client_secret

        if client_id == None or client_secret == None:
            raise Exception('Must set a client id and secret')

        client_creds  = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def getTokenHeader(self):
        client_creds_b64 = self.getClientCreds()
        return {
            'Authorization':f"Basic {client_creds_b64}"
        }

    def getTokenData(self):
        return {
            "grant_type":"client_credentials"
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.getTokenData()
        token_header = self.getTokenHeader()

        r = requests.post(token_url, data=token_data, headers=token_header)

        if r.status_code  not in range(200,299):
            raise Exception('Could not authenticate client.')
            #return False

        now = datetime.datetime.now()
        token_response_data = r.json()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now


        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now= datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}", 
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id ):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id ):
        return self.get_resource(_id, resource_type='artists')
    
    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists')
    
    
    def base_search(self, query_params): #type
        access_token = self.get_access_token()
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_URL = f"{endpoint}?{query_params}"
        r = requests.get(lookup_URL, headers=headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception('A query is required')
        if isinstance(query, dict):
            query = ' '.join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower() == 'not':
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f'{query} {operator} {operator_query}'
        query_params = urlencode({'q': query, 'type': search_type.lower()})
        return self.base_search(query_params)

In [ ]:
spotify = SpotifyAPI(client_id, client_secret)

# Top Weeks

# 1 radioactive

In [ ]:
radioactive = spotify.search({'track': 'Radioactive', 'artist': 'Imagine'}, search_type ='track')
radioactive

In [ ]:
test = pd.DataFrame.from_dict(radioactive,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

#  2 sail

In [ ]:
sail = spotify.search({'track': 'Sail', 'artist': 'AWOL'}, search_type ='track')
sail

In [ ]:
test = pd.DataFrame.from_dict(sail,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 3  I'm Yours

In [ ]:
Im_Yours = spotify.search({'track': 'Im Yours', 'artist': 'Jason'}, search_type ='track')
Im_Yours

In [ ]:
test = pd.DataFrame.from_dict(Im_Yours,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 4 Blinding Lights

In [ ]:
Blinding_Lights = spotify.search({'track': 'Blinding Lights', 'artist': 'The'}, search_type ='track')
Blinding_Lights

In [ ]:
test = pd.DataFrame.from_dict(Blinding_Lights,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 5 How Do I Live

In [ ]:
How_Do_I = spotify.search({'track': 'How Do I Live', 'artist': 'LeAnn'}, search_type ='track')
How_Do_I

In [ ]:
test = pd.DataFrame.from_dict(How_Do_I,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# Top Weeks @ #1

# 1 Old Town

In [ ]:
Old_Town = spotify.search({'track': 'Old Town Road', 'artist': 'Lil'}, search_type ='track')
Old_Town

In [ ]:
test = pd.DataFrame.from_dict(Old_Town,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 2 One Sweet Day

In [ ]:
One_Sweet = spotify.search({'track': 'One Sweet Day', 'artist': 'Mariah'}, search_type ='track')
One_Sweet

In [ ]:
test = pd.DataFrame.from_dict(One_Sweet,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 3 Despacito

In [ ]:
Despacito = spotify.search({'track': 'Despacito', 'artist': 'Luis'}, search_type ='track')
Despacito

In [ ]:
test = pd.DataFrame.from_dict(Despacito,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 4 Candle in the Wind 1997

In [ ]:
Candle = spotify.search({'track': 'Candle In The Wind 1997', 'artist': 'Elton'}, search_type ='track')
Candle

In [ ]:
test = pd.DataFrame.from_dict(Candle,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 5 Macarena

In [ ]:
Macarena_1 = spotify.search({'track': 'Macarena', 'artist': 'Los Del Rio'}, search_type ='track')
Macarena_1

In [ ]:
test = pd.DataFrame.from_dict(Macarena_1,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# Top Streams

# 1  Shape of You

In [ ]:
shape = spotify.search({'track': 'Shape of You', 'artist': 'Ed'}, search_type ='track')
shape

In [ ]:
test = pd.DataFrame.from_dict(shape,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 2 Blinding Lights

In [ ]:
Blinding_Lights = spotify.search({'track': 'Blinding Lights', 'artist': 'The'}, search_type ='track')
Blinding_Lights

In [ ]:
test = pd.DataFrame.from_dict(Blinding_Lights,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 3 Dance Monkey

In [ ]:
dance = spotify.search({'track': 'Dance Monkey', 'artist': 'Tones'}, search_type ='track')
dance

In [ ]:
test = pd.DataFrame.from_dict(dance,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 4 rockstar

In [ ]:
rockstar = spotify.search({'track': 'rockstar', 'artist': 'Post'}, search_type ='track')
rockstar

In [ ]:
test = pd.DataFrame.from_dict(rockstar,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# 5 Someone You Loved

In [ ]:
someone = spotify.search({'track': 'Someone You Loved', 'artist': 'Lewis'}, search_type ='track')
someone

In [ ]:
test = pd.DataFrame.from_dict(someone,orient='columns')
test

In [ ]:
test= test.tracks.apply(pd.Series)
test

In [ ]:
test2 = pd.DataFrame(test.iloc[1])
test2

In [ ]:
test3 = test2['items'].apply(pd.Series)
test3

# Top Weeks

In [ ]:
Title =  ['Radioactive', 'Sail', "I'm Yours", 'Blinding Lights', 'How Do I Live']
Popularity = [ 72, 74, 79, 90, 68]


df = pd.DataFrame({'Popularity' : Popularity }, index = Title)

ax = df.plot(kind='bar', figsize=(10,8), rot=0, title = 'Popularity Top Weeks on Chart', ylabel= 'Popularity')

ax.tick_params(
    axis="x", labelsize=12,
    labelcolor="black")
ax.tick_params(
    axis="y", labelsize=12,
    labelcolor="black")

sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

plt.show()

# 1 Weeks

In [ ]:
Title =  ['Old Town Road', 'One Sweet Day', 'Despacito', 'Candle in the Wind', 'Macarena']
Popularity = [78, 68, 80, 57, 60]


df = pd.DataFrame({'Popularity' : Popularity }, index = Title)

ax = df.plot(kind='bar', figsize=(10,8), rot=0, title = 'Popularity Top Weeks at #1', ylabel= 'Popularity')

ax.tick_params(
    axis="x", labelsize=12,
    labelcolor="black")
ax.tick_params(
    axis="y", labelsize=12,
    labelcolor="black")

sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

plt.show()

# Combined Chart Weeks

In [ ]:
group_labels =  ['Radioactive', 'Sail', "I'm Yours", 'Blinding Lights', 'How Do I Live']
Popularity = [ 72, 74, 79, 90, 68]

Weeks_on_Chart = [87, 79, 76, 76, 69]


df = pd.DataFrame({'Popularity': Popularity, 'Weeks on Chart': Weeks_on_Chart}, index = group_labels)

ax = df.plot(kind='bar', figsize=(10,8), rot=0, title = 'Average Headway, Dwell Time, Positive Adherence', ylabel= 'Mins & Adherence')

ax.tick_params(
    axis="x", labelsize=12,
    labelcolor="black")
ax.tick_params(
    axis="y", labelsize=12,
    labelcolor="black")

sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

plt.show()

# Combined Chart #1

In [ ]:
group_labels =  ['Old Town Road', 'One Sweet Day', 'Despacito', 'Candle in the Wind', 'Macarena' ]

Popularity_at_1= [78, 68, 80, 57, 60]

Weeks_at_1 = [19, 16, 16, 14, 14]

df = pd.DataFrame({'Popularity at 1': Popularity_at_1, 'Weeks at 1': Weeks_at_1}, index = group_labels)

ax = df.plot(kind='bar', figsize=(10,8), rot=0, title = 'Popularity and Weeks at 1', ylabel= 'Score')

ax.tick_params(
    axis="x", labelsize=12,
    labelcolor="black")
ax.tick_params(
    axis="y", labelsize=12,
    labelcolor="black")

sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

plt.show()

# Combined Chart

In [ ]:
group_labels =  ['Radioactive', 'Sail', "I'm Yours", 'Blinding Lights', 'How Do I Live', 'Old Town Road', 'One Sweet Day', 'Despacito', 'Candle in the Wind', 'Macarena' ]

Popularity = [ 72, 74, 79, 90, 68, 78, 68, 80, 57, 60]

Weeks_on_Chart = [87, 79, 76, 76, 69, 19, 16, 16, 14, 14]


df = pd.DataFrame({'Popularity': Popularity, 'Weeks on Chart': Weeks_on_Chart}, index = group_labels)

ax = df.plot(kind='bar', figsize=(18,8), rot=0, title = 'Popularity Weeks and #1', ylabel= 'Mins & Adherence')

ax.tick_params(
    axis="x", labelsize=12,
    labelcolor="black")
ax.tick_params(
    axis="y", labelsize=12,
    labelcolor="black")

sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

plt.show()


# Combined Chart Just Weeks

In [ ]:
Title =  ['Radioactive', 'Sail', "I'm Yours", 'Blinding Lights', 'How Do I Live', 'Old Town Road', 'One Sweet Day', 'Despacito', 'Candle in the Wind', 'Macarena']
Popularity = [ 72, 74, 79, 90, 68, 78, 68, 80, 57, 60]


df = pd.DataFrame({'Popularity' : Popularity }, index = Title)

ax = df.plot(kind='bar', figsize=(18,8), rot=0, title = 'Popularity and Weeks Combined', ylabel= 'Popularity')

ax.tick_params(
    axis="x", labelsize=12,
    labelcolor="black")
ax.tick_params(
    axis="y", labelsize=12,
    labelcolor="black")

sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

plt.show()